In [ ]:
import pandas as pd
from io import StringIO
import sklearn
import re
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import xml.etree.ElementTree as ET
import os
import ecg_plot

In [ ]:
filepath = '/groups/umcg-endocrinology/tmp02/projects/ukb-55495/data/metaData/v5/xml_T3/4399948_20205_3_0.xml'
tags = ['ObservationType', 'DICOMStudyUID', 'ReasonForStudy']
filepath2 = '/groups/umcg-endocrinology/tmp02/projects/ukb-55495/data/metaData/v5/xml_T3/3020947_20205_3_0.xml'

# df = pd.read_xml(filepath, xpath = '//root/{}'.format('/'.join(tags)))

In [ ]:
def getLabel():
    df = pd.read_xml(filepath, xpath = '//CardiologyXML/StripData/ArrhythmiaResults')
    label = df['BeatClass'].values[0]

In [ ]:
# df = pd.read_xml(filepath, xpath = '//CardiologyXML/ObservationDateTime')
# ObservationDateTime
# df = pd.read_xml(filepath, xpath = '//CardiologyXML/PatientVisit')
# df = pd.read_xml(filepath, xpath = '//CardiologyXML/PatientInfo')
# df = pd.read_xml(filepath, xpath = '//CardiologyXML/PatientInfo/BirthDateTime')
# df = pd.read_xml(filepath, xpath = '//MeasurementTable')
# df = pd.read_xml(filepath, xpath = '//RestingECGMeasurements')
# df = pd.read_xml(filepath, xpath = '//RestingECGMeasurements/MeasurementTable')
# df = pd.read_xml(filepath, xpath = '//CardiologyXML/RestingECGMeasurements/MeasurementTable/QDuration')  # to see measurements unit

# results! get labels from this data
# df = pd.read_xml(filepath, xpath = '//CardiologyXML/StripData/ArrhythmiaResults')
# df = pd.read_xml(filepath, xpath = '//ArrhythmiaResults')
# df = pd.read_xml(filepath, xpath = '//Diagnosis')
# df = pd.read_xml(filepath, xpath = '//Conclusion')
# resolution - one sample increment meets to this number of microVolts


# df = pd.read_xml(filepath, xpath = '//CardiologyXML/Interpretation/Conclusion') # not full description returns only last child
# df = pd.read_xml(filepath, xpath = '//CardiologyXML/StripData/WaveformData')
# df = pd.read_xml(filepath, xpath = '//CardiologyXML/StripData')

# different from CardiologyXML/StripData/WaveformData!!!
# df = pd.read_xml(filepath, xpath = '//CardiologyXML/RestingECGMeasurements/MedianSamples')
# df = pd.read_xml(filepath, xpath = '//CardiologyXML/RestingECGMeasurements/MedianSamples/WaveformData')


df = pd.read_xml(filepath, xpath = '//FullDisclosure')
df


In [ ]:
# fulldisclosure data looks wrong (not ecg)

df = pd.read_xml(filepath, xpath = '//FullDisclosure')
waves = df['FullDisclosureData'][0]
wave = re.sub(r"\s+", "", waves)
res = wave.split(',')
new = []
for i in range(len(res)):
    if res[i] != '':
        new.append(res[i])
res = [int(num) for num in new]
print(len(res))

# norm_res = normalize_waveform([res])
# ecg_plot.plot_1(norm_res[0], title = 'ECG 12')



In [ ]:
def get_raw_diagnosis(filepath):
    tree = ET.parse(filepath)
    root = tree.getroot()
    diagnosis = ''
    for inter in root.findall('Interpretation'):
        obj = inter.find('Diagnosis')
        if obj != None:
            for diag in obj:
                diagnosis += diag.text + '#'
    return diagnosis[:-1]


In [ ]:
def get_waves_from_file(filepath):
    xpath = '//RestingECGMeasurements/MedianSamples/WaveformData' #- contains the waveform data of the ecg strip for the given lead
    try:
        df = pd.read_xml(filepath, xpath=xpath)
    except ValueError as value_error:
        return []
    waves = get_waveform(df)
    normalized_waves = normalize_waveform(waves)
    return normalized_waves

def get_waveform(df, column='WaveformData'):
    waves = df[column]
    waves_processed = []
    for wave in waves:
        wave = re.sub(r"\s+", "", wave)
        res = [int(num) for num in wave.split(',')]
        waves_processed.append(list(res))
    return waves_processed

def normalize_waveform(waves):
    scaler = MinMaxScaler((-1, 1))
    return [scaler.fit_transform(np.array(wave).reshape(-1, 1)) for wave in waves]

def plot_waves_12(waves, columns = 4):
    ecg_plot.plot(waves, title = 'ECG 12', columns=columns)
    ecg_plot.show()

def save_waves(waves, filepath='images/ecg12.png', columns = 4):
    ecg_plot.plot(waves, title = 'ECG 12', columns=columns)
    ecg_plot.save_as_png(filepath, dpi = 500)

def reconstract_ECG(waves):
    return [np.array(list(wave.ravel()) * 5).reshape(-1, 1) for wave in waves]

In [ ]:
filepath1 = '/groups/umcg-endocrinology/tmp02/projects/ukb-55495/data/metaData/v5/xml_T3/4399948_20205_3_0.xml'
filepath2 = '/groups/umcg-endocrinology/tmp02/projects/ukb-55495/data/metaData/v5/xml_T3/3020947_20205_3_0.xml'

normalized_waves = get_waves_from_file(filepath2)
plot_waves_12(normalized_waves)

In [ ]:
long_waves = reconstract_ECG(normalized_waves)
plot_waves_12(long_waves, columns = 2)

In [ ]:
print(get_raw_diagnosis(filepath2))
print(get_raw_diagnosis(filepath1))

In [ ]:
def get_unique_diagnosis(order = 0):
    dir = '/groups/umcg-endocrinology/tmp02/projects/ukb-55495/data/metaData/v5/xml_T3/'
    short_label = []
    for file in os.listdir(dir):
        file_name = os.path.join(dir, file)
        d = get_raw_diagnosis(file_name)
        splittt = d.split('#')
        if len(splittt) > order:
            short_label.append(splittt[order])
    return set(short_label), len(short_label)

In [ ]:
diags0, len_origin = get_unique_diagnosis(0)
print(f'Found {len(diags0)} unique values out of {len_origin}')
print(diags0)

In [ ]:
def get_diagnosis_by_label(label):
    dir = '/groups/umcg-endocrinology/tmp02/projects/ukb-55495/data/metaData/v5/xml_T3/'
    short_label = []
    for file in os.listdir(dir):
        file_name = os.path.join(dir, file)
        d = get_raw_diagnosis(file_name)
        reversed_d = d[::-1]
        position = reversed_d.find('#')
        new_d = reversed_d[position+1:][::-1]
        if label in new_d:
            short_label.append(new_d)
    return set(short_label), len(short_label), short_label

In [ ]:
from collections import Counter
diags_normal, len_origin, origin = get_diagnosis_by_label('')

In [ ]:
for pair in Counter(origin).most_common(20):
    print(pair[0], "->", pair[1])

In [ ]:
normal_label = 'Normal ECG'
diags_normal, len_origin, origin = get_diagnosis_by_label(normal_label)
print(f'Found {len(diags_normal)} unique values out of {len_origin}')
print(diags_normal)

In [ ]:
normal_label = ' normal ECG'
diags_normal, len_origin, origin = get_diagnosis_by_label(normal_label)
print(f'Found {len(diags_normal)} unique values out of {len_origin}')
print(diags_normal)

In [ ]:
def get_dataset():
    normal_label = 'Normal ECG'
    dir = '/groups/umcg-endocrinology/tmp02/projects/ukb-55495/data/metaData/v5/xml_T3/'
    X, data_labels = [], []
    for file in sorted(os.listdir(dir)):
        file_name = os.path.join(dir, file)

        # get waves (X)
        normalized_12_waves = np.array(get_waves_from_file(file_name)).ravel()
        if len(normalized_12_waves) == 0:
            continue
        else:
            X.append(normalized_12_waves)

        # get labels (y)
        diagnosis = get_raw_diagnosis(file_name)
        reversed_diagnosis = diagnosis[::-1]
        position = reversed_diagnosis.find('#')
        new_diagnosis = reversed_diagnosis[position+1:][::-1]
        if normal_label in new_diagnosis:
            data_labels.append(0)
        else:
            data_labels.append(1)
    return X, data_labels

In [ ]:
X, y = get_dataset() # 0 - normal ECG, 1 - abnormal ECG

In [ ]:
X, y = np.array(X), np.array(y)   # 500 - number of samples per second
print(X.shape, y.shape)          # 2439 patients, 7200 = 12 leads * 600 measurements in time (600 - number of samples of one lead)

In [ ]:
def calculate_class_ratio(labels):
    normals, abnormals = 0, 0
    for label in labels:
        if label == 0:
            normals += 1
        else:
            abnormals += 1

    ratio = (abnormals // normals)
    print("normals:", normals, "abnormals:", abnormals)
    print("normals_to_abnormals_ration:", ratio)
    return ratio

In [ ]:
import random

def solve_imbalance_problem(X, labels):

    X = [np.append(X[i], labels[i]) for i in range(len(X))]
        
    ratio = calculate_class_ratio(labels)

    x_new = []
    for x in X:
        if x[-1] == 0:
            for i in range(ratio):
                x_new.append(x)


    for xx in x_new:
        X.append(xx)
    random.shuffle(X)

    updated_labels = []

    # separate waves and labels again
    for i in range(len(X)):
        updated_labels.append(X[i][-1])
        X[i] = X[i][:-1]
    
    calculate_class_ratio(updated_labels)

    return X, updated_labels

In [ ]:
training_proportion = int(0.75 * len(X))

training_set_X = X[:training_proportion]
testing_set_X = X[training_proportion:]
training_set_y = y[:training_proportion]
testing_set_y = y[training_proportion:]

train_X_balanced, train_y_balanced = solve_imbalance_problem(training_set_X, training_set_y)

In [ ]:
def reverse_one_hot(predictions):
    reversed_x = []
    for x in predictions:
        reversed_x.append(np.argmax(np.array(x)))
    return reversed_x

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

image_shape = (24, 300)
# image_shape = (12, 600)
# image_shape = (1, 600*12) # does not work

def learn_the_model(training_set_X, training_set_y, testing_set_X, testing_set_y):

    test_labels = tf.keras.utils.to_categorical(testing_set_y, 2)
    train_labels = tf.keras.utils.to_categorical(training_set_y, 2)
    
    train_images = training_set_X.reshape(training_set_X.shape[0], image_shape[0], image_shape[1], 1)
    test_images = testing_set_X.reshape(testing_set_X.shape[0], image_shape[0], image_shape[1], 1)
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_shape[0], image_shape[1], 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adadelta(), metrics=['accuracy', 'mae', 'mse'])
    
    train_data_size = train_images.shape[0]
    test_data_size = test_images.shape[0]
    
    print("model will be trained with {} and be tested with {} sample".format(train_data_size, test_data_size))
    print("Fitting model to the training data...")
    model.fit(train_images, train_labels, batch_size=150, epochs=5, verbose=1, validation_data=None)
    
    predictions = model.predict(test_images, batch_size=150, verbose=1)
    print(model.evaluate(test_images, test_labels, batch_size=150, verbose=1))
    
    # print("Train conf matrix: ", confusion_matrix(np.array(reverse_one_hot(train_labels)),
    #                                                   np.array(reverse_one_hot(
    #                                                       model.predict(train_images, batch_size=150,
    #                                                                     verbose=1)))))
    # print("Test conf matrix: ", confusion_matrix(np.array(reverse_one_hot(test_labels)),
    #                                                  np.array(reverse_one_hot(predictions))))
    
    print("Evaluation accuracy score = ", accuracy_score(predictions, testing_set_y))
    print("Confusion matrix = ", confusion_matrix(testing_set_y, predictions))

In [ ]:
learn_the_model(training_set_X, training_set_y, testing_set_X, testing_set_y)

In [ ]:
xpath = '//Sagittal' 
df = pd.read_xml(filepath, xpath=xpath)

In [ ]:
df

In [ ]:
xpath = '//FullDisclosureData' 
df = pd.read_xml(filepath, xpath=xpath)

In [ ]:
df

In [ ]:
# split '/groups/umcg-endocrinology/tmp02/projects/ukb-55495/data/metaData/v5/ukb674725_headerOfsetFixed_4ColsRemoved_humanreadable.txt' files into smaller files to explore the content

filepath = '/home/umcg-asorova/project/myDataSplit50/xaa'
df = pd.read_csv(filepath, sep="\t", engine='python',quotechar=' ')

# count missing values and remove column with more than 0.2% of missing values : from 31110 columns result in 309
# percentage_missing = df.isnull().sum().sort_values(ascending=False)/len(df)*100
# for column in percentage_missing.keys().tolist():
#     if percentage_missing[column] > 0.2:
#         df.drop(column, inplace = True, axis=1)
# df.head()

In [ ]:
ids = df['eid']
print

In [ ]:
print(list(df.columns))

In [ ]:
df['Vascular/heart problems diagnosed by doctor_0.0'].head()

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

# oversampling technique
sm = SMOTE(random_state=40, n_jobs=8, sampling_strategy='not majority')
X_train_resampled, y_train_resampled = sm.fit_resample(training_set_X, training_set_y)

print('Original dataset shape:', Counter(training_set_y))
print('Resampled dataset shape:', Counter(y_train_resampled))

In [ ]:
learn_the_model(X_train_resampled, y_train_resampled, testing_set_X, testing_set_y)

In [ ]:
from imblearn.over_sampling import ADASYN

# oversampling technique
sm = ADASYN(random_state=40, n_jobs=8, sampling_strategy='not majority')
X_train_resampled_ADASYN, y_train_resampled_ADASYN = sm.fit_resample(training_set_X, training_set_y)

print('Original dataset shape:', Counter(training_set_y))
print('Resampled dataset shape:', Counter(y_train_resampled_ADASYN))

In [ ]:
learn_the_model(X_train_resampled_ADASYN, y_train_resampled_ADASYN, testing_set_X, testing_set_y)

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from imblearn.under_sampling import ClusterCentroids

# undersampling technique
cc = ClusterCentroids(
    estimator=MiniBatchKMeans(n_init=1, random_state=0), random_state=42, sampling_strategy='not minority'
)
X_train_resampled_CC, y_train_resampled_CC = cc.fit_resample(training_set_X, training_set_y)
print('Original dataset shape:', Counter(training_set_y))
print('Resampled dataset shape:', Counter(y_train_resampled_CC))

In [ ]:
learn_the_model(X_train_resampled_CC, y_train_resampled_CC, testing_set_X, testing_set_y)